<center>
<h1>Chapter Two</h1>
</center>

<hr>

#I. Bring in data

I'm going to try out a new package that gives a lot of details on a pandas dataset.

In [ ]:
%%capture
! pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip 

In [ ]:
from pandas_profiling import ProfileReport

In [ ]:
import pandas as pd

In [ ]:
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQjIoC4LWfiuFmnix6TUAGSOPK29QMsn5Sb9DS73mNmMqikngNSG9ntmxHUO7ySZXTPpmoP8yAV0auz/pub?output=csv'  #plain old string I got from Google Sheets
#url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRMqzd1C4Wzh-TyBhmZVARM4K99jYnav0tCYo_fb-nArlz9T6TDSszP-vKJ2AD3W3AsXUBaXVCtynaO/pub?output=csv'
titanic_table = pd.read_csv(url)  #using our new package to read in an entire dataset - the coolest

In [ ]:
titanic_table.head()  #print first 5 rows of the table

,Name,Age,Gender,Class,Joined,Married,Survived,Fare,Bio,Occupation,Class/Dept,Cabin,Boat,Nationality,URL
0,"ABBING, Mr Anthony",41.0,Male,C3,Southampton,0.0,0,7.0,"anthony abbing was born in cincinnati,...",Blacksmith,3rd Class Passenger,NaN,NaN,American,https://www.encyclopedia-titanica.org/titanic-...
1,"ABBOTT, Mr Ernest Owen",21.0,Male,Crew,Southampton,0.0,0,0.0,ernest owen abbott was born in southam...,Lounge Pantry Steward,Victualling Crew,NaN,NaN,English,https://www.encyclopedia-titanica.org/titanic-...
2,"ABBOTT, Mr Eugene Joseph",13.0,Male,C3,Southampton,NaN,0,20.0,"\tmaster eugene joseph abbott, 13, w...",Scholar,3rd Class Passenger,NaN,NaN,American,https://www.encyclopedia-titanica.org/titanic-...
3,"ABBOTT, Mr Rossmore Edward",16.0,Male,C3,Southampton,0.0,0,NaN,"mr rossmore edward abbott, 16, of prov...",Jeweller,3rd Class Passenger,NaN,NaN,"English, American",https://www.encyclopedia-titanica.org/titanic-...
4,"ABELSON, Mr Samuel",NaN,Male,C2,Cherbourg,0.0,0,24.0,"mr samuel abelson, 30, from russia, an...",NaN,2nd Class Passenger,NaN,NaN,Russian,https://www.encyclopedia-titanica.org/titanic-...


In [ ]:
profile = ProfileReport(titanic_table, title='Titanic data', html={'style': {'full_width':True}})

In [ ]:
profile.to_notebook_iframe()  #need this to work within colab

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

###How to use all this data

We will use some of it. In particular, we will look at ways of filling missing values. We will also look at the correlation information. You worked on dcor in chapter 1. We will also look at Pearson.


#II. Let's start wrangling
<img src='https://www.dropbox.com/s/9fcc1crlxp19ijt/major_section.png?raw=1' width='300'>

Our goal is to use machine learning methods to make predictions. For now we will be predicting survival for Titanic passengers. But we will also look at a  dataset formed around predicting the satisfaction of cable customers.

I've cheated and looked ahead. I know what form we need our data in to use machine learning algorithms. And I can tell you we have a ways to go to get there. I'll refer to getting our data in shape as data wrangling or just wrangling.

I'll take it a step at a time, using a chapter to look at a specific wrangling step.

##Features versus label

It may look a little weird now, and make more sense when we get to modeling, but I need to break up the raw table into 2 pieces: feature columns and label column. We will be wrangling the feature columns but not the label column.

The label column is the one we will choose to predict. We generally will not wrangle it.

So let's split it out.

In [ ]:
titanic_features = titanic_table.drop(columns='Survived')
titanic_label = titanic_table['Survived'].to_list()  #we won't use this for several more chapters

#III. Dropping columns

I'm going to make a decision at this point to reduce the columns we consider. This is an important decision and could bite me later if I remove columns that are valuable in predicting. If this was not an academic exercise, you would likely need to justify why you drop columns - someone will ask.

There are 2 ways to go: (1) I could use a pandas method that removes columns or (2) I can just say what columns I want to keep.

In [ ]:
print(titanic_features.columns.to_list())

['Name', 'Age', 'Gender', 'Class', 'Joined', 'Married', 'Fare', 'Bio', 'Occupation', 'Class/Dept', 'Cabin', 'Boat', 'Nationality', 'URL']


In [ ]:
#Way 1: columns I want to remove

titanic_df_1 = titanic_features.drop(columns=['Name',  'Bio', 'Occupation', 'Class/Dept', 'Cabin', 'Boat', 'Nationality', 'URL']
)

In [ ]:
#Way 2: Say what columns I want to keep

titanic_df_keep = titanic_features[['Age', 'Gender', 'Class', 'Joined', 'Married',  'Fare']]

###Convention note

pandas refers to tables as *DataFrames*, nicknamed `df`. I sometimes use `df` instead of table - easier to type :)

#IV. Transforming string values to numeric

Few machine learning algorithms can work with string values. Let's look again at the data we have to spot string values.

In [ ]:
titanic_df_1.dtypes

Age        float64
Gender      object
Class       object
Joined      object
Married    float64
Fare       float64
dtype: object

### The datatype `object` is typically a string

It is what pandas uses to signal a non-numeric column.

###We have 3 string columns

How we transform is going to take some thought. We need to make them all numeric.

###Let's start with `Gender`

What makes this relatively easy is that it is a binary column: only has 2 unique values. In cases like this, a simple replace method will do the work.

I am going to aribtrarily replace `Male` with `0` and `Female` with `1`.

I'll need a new pandas method.

In [ ]:
column = 'Gender'
mapping = {'Male':0, 'Female':1}
titanic_df_2 = titanic_df_1.copy()  #I'm keeping versions so I can roll back

titanic_df_2[column].replace(mapping, inplace=True)
titanic_df_2.tail()

,Age,Gender,Class,Joined,Married,Fare
2203,2.0,1,C3,Cherbourg,0.0,22.0
2204,23.0,1,C3,Cherbourg,1.0,22.0
2205,22.0,1,C3,Southampton,0.0,7.0
2206,22.0,1,C1,Southampton,0.0,61.0
2207,27.0,0,C3,Cherbourg,0.0,7.0


##Reminder to self

Clear off the mural site before next problem.

##I'll set up a little test dataset for you

In [ ]:
test_data = {'Temperature':['Medium', 'High', 'High', 'Low'],
             'Rainfall': ['None', 'Light', 'Heavy', 'Light'],
             'Region': ['Desert', 'Tropical', 'Desert', 'Tropical'],
             'Reporting_ID': ['foo', 'fum', 'foe', 'fie']
            }
test_table = pd.DataFrame(test_data)
test_table

,Temperature,Rainfall,Region,Reporting_ID
0,Medium,None,Desert,foo
1,High,Light,Tropical,fum
2,High,Heavy,Desert,foe
3,Low,Light,Tropical,fie


<img src='https://www.dropbox.com/s/8x575mvbi1xumje/cash_line.png?raw=1' height=3 width=500><br>
<img src='https://www.gannett-cdn.com/-mm-/56cbeec8287997813f287995de67747ba5e101d5/c=9-0-1280-718/local/-/media/2018/02/15/Phoenix/Phoenix/636542954131413889-image.jpg' height=50 align=center>  Create `test_table_2` that focuses on `Region`. Transform it to a numerical column

Take a screenshot of your code and put it on the mural site: https://app.mural.co/invitation/mural/cis4236034/1639511781261?sender=u173416b87b3b8c1cbe335790&key=3993cb03-56a9-4fe5-b4d9-6304e32ff459.




In [ ]:
column = 
mapping = 
test_table_2 = test_table.copy()
test_table_2 = 
test_table_2

## Ordinal columns

We can view a column as ordinal if it has a set of string values where there is an ordering among the values. If the column contained temperature gradients, it is an ordinal column. I can say one value, e.g., 'High', has an arithmetic relationship to another, e.g., 'Low'. So asking High>Low makes sense.

I can't say this is always a slam-dunk decision. I'm looking at you, `Class` column. I am going to make the following argument: from what I know of the social strata at the time (1912), first class passengers (C1) were viewed with more respect than lower classes (C2, C3). And Crew were the lowest of all. So I am going to view Class as ordinal. But I would not be surprised if you want to pushback on this. It is a bit flimsy.

The good news is that we can use same replace mechanism to work with orindal columns. Check it out.

In [ ]:
column = 'Class'
mapping = {'Crew':0,
            'C3': 1,
            'C2': 2,
            'C1': 3}
titanic_df_3 = titanic_df_2.copy()  #I'm keeping versions so I can roll back

titanic_df_3[column].replace(mapping, inplace=True)
titanic_df_3.tail()

,Age,Gender,Class,Joined,Married,Fare
2203,2.0,1,1.0,Cherbourg,0.0,22.0
2204,23.0,1,1.0,Cherbourg,1.0,22.0
2205,22.0,1,1.0,Southampton,0.0,7.0
2206,22.0,1,3.0,Southampton,0.0,61.0
2207,27.0,0,1.0,Cherbourg,0.0,7.0


Again, this is a subjective decision. I'm thinking ahead to the type of machine learning algorithms we will use. They will ask how close a crewman is to a first class passenger (for instance). I view that difference as big so gave them a wide separation, e.g., `abs(0-3)`. In comparison, I view the separation from a crewman and a third class passenger as small.

##Reminder to self

Clear off the mural site before next problem.

<img src='https://www.dropbox.com/s/8x575mvbi1xumje/cash_line.png?raw=1' height=3 width=500><br>
<img src='https://www.gannett-cdn.com/-mm-/56cbeec8287997813f287995de67747ba5e101d5/c=9-0-1280-718/local/-/media/2018/02/15/Phoenix/Phoenix/636542954131413889-image.jpg' height=50 align=center>  Transform the `Temperature` and `Rainfall` columns into numerical form.

Take a screenshot of your code and put it on the mural site: https://app.mural.co/invitation/mural/cis4236034/1639511781261?sender=u173416b87b3b8c1cbe335790&key=3993cb03-56a9-4fe5-b4d9-6304e32ff459.




In [ ]:
#Reminder of what is in test_table
test_table

,Temperature,Rainfall,Region,Reporting_ID
0,Medium,None,Desert,foo
1,High,Light,Tropical,fum
2,High,Heavy,Desert,foe
3,Low,Light,Tropical,fie


In [ ]:
#Make changes to columns in new version of table


## Nominal columns

We can view a column as nominal if it has a set of string values where there is no interesting ordering among the values. If the column contained baby names, it is a nominal column. I can't say one value, e.g., 'Emily', has an arithmetic relationship to another, e.g., 'Sara'. So asking `Emily>Sara` makes no sense.

The replace method I used for `Class` does not work well for nominal columns. If I simply gave each baby name a number, then I am introducing an implied ordering. All of sudden, Emily might be way more important than Sara.

We need to think about other approaches. We still want to replace string values with numbers. But avoid introducing orderings that make no sense.

Our focus will be on the `Joined` column.

### Dealing with the nominal column `Joined`

There has been a recent uptick in interest in ways to deal with nominal columns. I'll look at the standard approach.


In [ ]:
set(titanic_df_3['Joined'])

{'Belfast', 'Cherbourg', 'Queenstown', 'Southampton'}

##One Hot Encoding (or dummy encoding from Statistics)

The general idea is as follows.

1. The Joined column has 4 unqiue values. So I will add 4 new columns to the table: `Joined_Belfast, Joined_Cherbourg, Joined_Queenstown`, and `Joined_Southampton`.

2. I'll set all their values to 0 initially.

3. For a specific row, if that row has a value of `Belfast` in `Joined`, I'll reset the `Joined_Belfast` value to 1 (and let 0s remain in the other 3). Do the same for other `Joined` values.

4. What I end up with for each row is exactly a single 1 in the 4 new columns and 0 otherwise. I have one-hot encoded `Joined` in this way. The `Joined` column is dropped at this point without losing any information.

I suppose I could write a loop to do this, but pandas has a method built-in to do it. Check it out.



In [ ]:
titanic_df_4 = pd.get_dummies(titanic_df_3,
                               prefix='Joined',    #your choice
                               prefix_sep='_',     #your choice
                               columns=['Joined'],
                               dummy_na=False,    #will try to impute later so leave NaNs in place
                               drop_first=False    #really should be True but I wanted to give a clearer picture
                               )

titanic_df_4.head()

,Age,Gender,Class,Married,Fare,Joined_Belfast,Joined_Cherbourg,Joined_Queenstown,Joined_Southampton
0,41.0,0,1.0,0.0,7.0,0,0,0,1
1,21.0,0,0.0,0.0,0.0,0,0,0,1
2,13.0,0,1.0,NaN,20.0,0,0,0,1
3,16.0,0,1.0,0.0,NaN,0,0,0,1
4,NaN,0,2.0,0.0,24.0,0,1,0,0


Two of the arguments are less than intuitive. Here is description from docs:
<pre>
dummy_na bool, default False
Add a column to indicate NaNs, if False NaNs are ignored.

drop_first bool, default False
Whether to get k-1 dummies out of k categorical levels by removing the first level.
</pre>
For latter in particular, we can leave one out and infer it from all 0 values in others. Recommended to avoid colinearity issues. So let's do it.

In [ ]:
titanic_df_4 = pd.get_dummies(titanic_df_3,
                               prefix='Joined',    #your choice
                               prefix_sep='_',     #your choice
                               columns=['Joined'],
                               dummy_na=False,    #will try to impute later so leave NaNs in place
                               drop_first=True    #will drop Belfast and infer it
                               )

titanic_df_4.head()

,Age,Gender,Class,Married,Fare,Joined_Cherbourg,Joined_Queenstown,Joined_Southampton
0,41.0,0,1.0,0.0,7.0,0,0,1
1,21.0,0,0.0,0.0,0.0,0,0,1
2,13.0,0,1.0,NaN,20.0,0,0,1
3,16.0,0,1.0,0.0,NaN,0,0,1
4,NaN,0,2.0,0.0,24.0,1,0,0


##Reminder to self

Clear off the mural site before next problem.

<img src='https://www.dropbox.com/s/8x575mvbi1xumje/cash_line.png?raw=1' height=3 width=500><br>
<img src='https://www.gannett-cdn.com/-mm-/56cbeec8287997813f287995de67747ba5e101d5/c=9-0-1280-718/local/-/media/2018/02/15/Phoenix/Phoenix/636542954131413889-image.jpg' height=50 align=center>  Transform the `Reporting_ID` column into numerical form. Drop the first.

Take a screenshot of your code and put it on the mural site: https://app.mural.co/invitation/mural/cis4236034/1639511781261?sender=u173416b87b3b8c1cbe335790&key=3993cb03-56a9-4fe5-b4d9-6304e32ff459.




In [ ]:
#Reminder of what is in test_table
test_table

,Temperature,Rainfall,Region,Reporting_ID
0,Medium,None,Desert,foo
1,High,Light,Tropical,fum
2,High,Heavy,Desert,foe
3,Low,Light,Tropical,fie


In [ ]:
#Make changes to column in new version of table


##"Dirty" category columns

Take a look at the unique values in the `Occupation` column.

In [ ]:
len(set(titanic_features['Occupation']))

385

That is a lot of values to one hot encode! We will add 384 new columns. A bit much. We do not have to deal with it because we dropped it. But here is an article that gives you some ways to deal with a Nominal column with large numbers of values.

http://dirtydata.science/python/gen_notes/02_dirty_categories.html

In [ ]:
range(10)

range(0, 10)

In [ ]:
list(range(10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [ ]:
zip([1,2,3],[4,5,6])

In [ ]:
list(zip([1,2,3],[4,5,6]))

[(1, 4), (2, 5), (3, 6)]

In [ ]:
x = (4,5,6)
x.append(7)

AttributeError: ignored

In [ ]:
y = [1,2,3]
y.append(4)

In [ ]:
y

[1, 2, 3, 4]

In [ ]:
list(x)

[4, 5, 6]

In [ ]:
set(x)

{4, 5, 6}

In [ ]:
[i for i in range(10)]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [ ]:
[int(z) for z in [.3,.4,5]]

[0, 0, 5]

#Stop for challenges

I'd like you to work on applying what you have learned above to a new dataset. We will be using this dataset for the remainder of the course. Check it out.

In [ ]:
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQPM6PqZXgmAHfRYTcDZseyALRyVwkBtKEo_rtaKq_C7T0jycWxH6QVEzTzJCRA0m8Vz0k68eM9tDm-/pub?output=csv'


In [ ]:
import pandas as pd
customers_df = pd.read_csv(url)
customers_df.head()

,ID,Gender,Experience Level,Time Spent,OS,ISP,Age,Rating
0,3,Female,medium,NaN,iOS,Xfinity,NaN,0
1,27,Male,medium,71.97,Android,Cox,50.0,0
2,30,Female,medium,101.81,NaN,Cox,49.0,1
3,40,Female,medium,86.37,Android,Xfinity,53.0,0
4,52,Female,medium,103.97,iOS,Xfinity,58.0,0


In [ ]:
customers_df.tail()

##Summary

Each row represents a customer (know by their ID). It gives the amount of time each spent trying to get a phone app connected to their ISP provider. The Rating is what the customer rated the help he or she got.

In [ ]:
profile = ProfileReport(customers_df, title='Cable data', html={'style': {'full_width':True}})
profile.to_notebook_iframe()  #need this to work within colab

#Challenge 0 (no turn in but needed for following challenges)

* First drop the ID column. It carries no useful info.

* Second encode the string columns. Leave `NaN` values in place - we will deal with them later.

* Make sure to separate out Ordinal and Nominal columns: they each have their own way of being encoded.

* For nominal columns, go ahead and use `drop_first=True`.

You do not have to save your transformed table to file.

You can ignore the label column for now.

In [ ]:
#I'll get you started
customer_features = customers_df.drop(columns='Rating')  #concentrate on this
customer_label = customers_df['Rating'].to_list()  #you can ignore this column for now

Reminder of what table looks like.



In [ ]:
customer_features.head()

###Determining unique values in column quickly

You could look at the detailed profile above. But here is another way.

In [ ]:
set(customers_df['Gender'])  #unique values in a column

###Ok, go to it


In [ ]:
#your final table should end up in wrangled_df_1


In [ ]:
wrangled_df_1.describe(include='all').T

##What I end up with

<img src='https://www.dropbox.com/s/78ickq1qjwyp87y/Screen%20Shot%202021-10-18%20at%2011.53.57%20AM.png?raw=1' height=150>

`describe(include='all').T`:

<img src='https://www.dropbox.com/s/nqaw1u3rc29hr6y/Screen%20Shot%202021-09-29%20at%201.13.40%20PM.png?raw=1' height=150>


#Start here next class period

#X. Can we normalize wrangling?

What is appealing to me (and the Data Science community) is the idea of data pipelines.


<img src='https://hazelcast.com/wp-content/uploads/2019/11/24_DataPipleline-1.png'>

The general idea is you pass your original dataframe in on the left and then perform a set of wrangling operations on it, producing the final dataframe version on the right. Each operation is one step that takes a dataframe in and produces a dataframe.

It turns out `sklearn` has us covered. If you follow their guidelines, you can build a data pipeline like above. The general idea is that the above operations become calls on a method called `transform`. That method is attached to a transformer class. Each operation above is then an instance of a transformer class where its `transform` method is called. As you might guess, the `transform` method takes in a dataframe and produces a dataframe.

##We have a slight problem - `sklearn` is missing transformers we want

Here are the 2 transformers I think we want:

1. A mapping transformer that will replace values in a column with other values, i.e., do a mapping.

2. A one-hot encoding transformer that will take a column and replace it with new columns that reflect the column values.

And we would like both these transformers to stick with dataframes: they accept a dataframe; they produce a dataframe.

Unfortunately, `sklearn` does not have these 2 transformers in its library. But there is hope! `sklearn` gives us the ability to build our own customer transformers and still fit them in a pipeline it recognizes.

Let's go over the code to build one of these missing transformers, the one that does mapping.



###Important note:

It's easy to get tripped up here in OO land. What we want is a general mapping class that can be instantiated to work on specific column mappings.

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin #gives us the tools to build custom transformers

#This class maps values in a column, numeric or categorical.
class MappingTransformer(BaseEstimator, TransformerMixin):
  
  def __init__(self, mapping_column, mapping_dict:dict):  
    self.mapping_dict = mapping_dict
    self.mapping_column = mapping_column  #column to focus on

  def fit(self, X, y = None):
    print(f"Warning: MappingTransformer.fit does nothing.")
    return X

  def transform(self, X):
    assert isinstance(X, pd.core.frame.DataFrame), f'MappingTransformer.transform expected Dataframe but got {type(X)} instead.'
    assert self.mapping_column in X.columns.to_list(), f'MappingTransformer.transform unknown column {self.mapping_column}'
    X_ = X.copy()
    X_[self.mapping_column].replace(self.mapping_dict, inplace=True)
    return X_

  def fit_transform(self, X, y = None):
    result = self.transform(X)
    return result

In [ ]:
foo = MappingTransformer('fum', {})
foo.fit(5)

5

###The initializer `__init__`

When we instantiate the class, we need to decide what info we need. In this case, we need to know what column to work on and the mapping. I am using a dictionary to denote the mapping. 
<pre>
class MappingTransformer(BaseEstimator, TransformerMixin):
  def __init__(self, mapping_column, mapping_dict:dict):  
    self.mapping_dict = mapping_dict
    self.mapping_column = mapping_column  #column to focus on
</pre>
Note I am using 2 of sklearn's classes as super-classes: `class MappingTransformer(BaseEstimator, TransformerMixin)`.

###The `fit` process

It's a bit confusing. Each transformer needs at least 3 methods: `fit, transform, fit_transform`. There is a `fit` step because some transformers have to analyze/train on data before getting to doing transformations. We will see that later. But in our case, there is no `fit` step needed. So if someone invokes `fit`, we do nothing and give a warning.
<pre>
  def fit(self, X, y = None):
    print("Warning: MappingTransformer.fit does nothing.")
    return X
</pre>

###The `transform` process

This is where the action is. We will take a table in and do the mapping on the designated column (see constructor).
<pre>
  def transform(self, X):
    assert isinstance(X, pd.core.frame.DataFrame), f'MappingTransformer.transform expected Dataframe but got {type(X)} instead.'
    assert self.mapping_column in X.columns.to_list(), f'MappingTransformer.transform unknown column {self.mapping_column}'
    X_ = X.copy()
    X_[self.mapping_column].replace(self.mapping_dict, inplace=True)
    return X_
</pre>
Note I am making a copy of the input. This is good practice. A new table is returned with mapping done.

###The `fit_transform` process

This is a convenience function that combines the 2 methods. In our case, we simply skip `fit` and call `transform` directly.
<pre>
  def fit_transform(self, X, y = None):
    result = self.transform(X)
    return result
</pre>


###Test it out

Let's our new transformer to `Gender` and `Class`. Reminder: we will need to instantiate the class twice to get two separate operations in our pipeline.

In [ ]:
import pandas as pd

In [ ]:
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQjIoC4LWfiuFmnix6TUAGSOPK29QMsn5Sb9DS73mNmMqikngNSG9ntmxHUO7ySZXTPpmoP8yAV0auz/pub?output=csv'  #plain old string I got from Google Sheets
titanic_table = pd.read_csv(url)

In [ ]:
titanic_features = titanic_table.drop(columns='Survived')
titanic_features.head()  #print first 5 rows of the table

In [ ]:
gender_transformer = MappingTransformer('Gender', {'Male': 0, 'Female': 1})  #produce an operator
X2_df = gender_transformer.fit_transform(titanic_features)  #apply the operator

In [ ]:
X2_df.head()

In [ ]:
class_transformer = MappingTransformer('Class', {'Crew': 0, 'C3': 1, 'C2': 2, 'C1': 3})
X3_df = class_transformer.transform(X2_df)

In [ ]:
X3_df.head()

##Late breaking news!!!

I was pondering a way to avoid writing out class name below:
<pre>
"Warning: MappingTransformer.fit does nothing."

f'MappingTransformer.transform expected Dataframe but got {type(X)} instead.'

f'MappingTransformer.transform unknown column {self.mapping_column}'
</pre>
Is there a way to get the class name, e.g., `MappingTransformer`, programmatically? Yes. We can use
<pre>
self.__class__.__name__
</pre>
So now I can have:
<pre>
f"Warning: {self.__class__.__name__}.fit does nothing.")
f'{self.__class__.__name__}.transform expected Dataframe but got {type(X)} instead.'
f'{self.__class__.__name__}.transform unknown column {self.mapping_column}'
</pre>
I like this better. I did not change the code above but I will start using this in future.

#VI. Ready for pipeline

We can define a pipeline that will call our transformers, in order, to pass the data through. If we start with `titanic_features`, the final output will be data in a transformed form.

Let me show you how to define a pipeline. It is relatively straightforward.

In [ ]:
from sklearn.pipeline import Pipeline

#first define the pipeline
titanic_transformer = Pipeline(steps=[
    ('gender', MappingTransformer('Gender', {'Male': 0, 'Female': 1})),
    ('class', MappingTransformer('Class', {'Crew': 0, 'C3': 1, 'C2': 2, 'C1': 3})),
    ], verbose=True)

#now invoke it
transformed_df = titanic_transformer.fit_transform(titanic_features)

You can see that each operation is a tuple. First item is name you give the operation. This name is your choice. Second is call on a constructor to give you a transformer object.

Once you invoke it, each transformer is built and then called in turn, passing its output to next transformer.

In [ ]:
transformed_df.head()

We got what we wanted!


##Reminder to self

Clear off the mural site before next problem.

<img src='https://www.dropbox.com/s/8x575mvbi1xumje/cash_line.png?raw=1' height=3 width=500><br>
<img src='https://www.gannett-cdn.com/-mm-/56cbeec8287997813f287995de67747ba5e101d5/c=9-0-1280-718/local/-/media/2018/02/15/Phoenix/Phoenix/636542954131413889-image.jpg' height=50 align=center>  I'm going to let you work from scratch on a new transformer called RenamingTransformer. This transformer will rename one or more columns. First figure out how to rename columns in pandas.

Take a screenshot of your code and put it on the mural site: https://app.mural.co/invitation/mural/cis4236034/1639511781261?sender=u173416b87b3b8c1cbe335790&key=3993cb03-56a9-4fe5-b4d9-6304e32ff459.




In [ ]:
#First, figure out how to rename a column in pandas
#Try renaming URL to More_Information and Boat to Life_Boat.


##Reminder to self

Clear off the mural site before next problem.

###Here is MappingTransformer repeated for reference

<pre>
#This class maps values in a column, numeric or categorical.
class MappingTransformer(BaseEstimator, TransformerMixin):
  
  def __init__(self, mapping_column, mapping_dict:dict):  
    self.mapping_dict = mapping_dict
    self.mapping_column = mapping_column  #column to focus on

  def fit(self, X, y = None):
    print("Warning: MappingTransformer.fit does nothing.")
    return X

  def transform(self, X):
    assert isinstance(X, pd.core.frame.DataFrame), f'MappingTransformer.transform expected Dataframe but got {type(X)} instead.'
    assert self.mapping_column in X.columns.to_list(), f'MappingTransformer.transform unknown column {self.mapping_column}'
    X_ = X.copy()
    X_[self.mapping_column].replace(self.mapping_dict, inplace=True)
    return X_

  def fit_transform(self, X, y = None):
    result = self.transform(X)
    return result
</pre>

<img src='https://www.dropbox.com/s/8x575mvbi1xumje/cash_line.png?raw=1' height=3 width=500><br>
<img src='https://www.gannett-cdn.com/-mm-/56cbeec8287997813f287995de67747ba5e101d5/c=9-0-1280-718/local/-/media/2018/02/15/Phoenix/Phoenix/636542954131413889-image.jpg' height=50 align=center>  Ok, write the `__init__` method.

Take a screenshot of your code and put it on the mural site: https://app.mural.co/invitation/mural/cis4236034/1639511781261?sender=u173416b87b3b8c1cbe335790&key=3993cb03-56a9-4fe5-b4d9-6304e32ff459.




In [ ]:
#This class will rename one or more columns.
class RenamingTransformer(BaseEstimator, TransformerMixin):
  #First write __init__ method.
  #Hint: maybe copy and paste from MappingTransformer here then fix up for new problem?



In [ ]:
#might want to test your class here.
#try creating an object from it.


##Reminder to self

Clear off the mural site before next problem.

<img src='https://www.dropbox.com/s/8x575mvbi1xumje/cash_line.png?raw=1' height=3 width=500><br>
<img src='https://www.gannett-cdn.com/-mm-/56cbeec8287997813f287995de67747ba5e101d5/c=9-0-1280-718/local/-/media/2018/02/15/Phoenix/Phoenix/636542954131413889-image.jpg' height=50 align=center>  Ok, write the `transform` method. Ignore asserts for now.

Take a screenshot of your code and put it on the mural site: https://app.mural.co/invitation/mural/cis4236034/1639511781261?sender=u173416b87b3b8c1cbe335790&key=3993cb03-56a9-4fe5-b4d9-6304e32ff459.




In [ ]:
#This class will rename one or more columns.
class RenamingTransformer(BaseEstimator, TransformerMixin):
  #your __init__ method below

  #write the transform method without asserts. Again, maybe copy and paste from MappingTransformer and fix up.



In [ ]:
#might want to test here.
#Try renaming URL to More_Information and Boat to Life_Boat using your transformer.


##Reminder to self

Clear off the mural site before next problem.

##Look at asserts for transform

In [ ]:
#First one I'll give you. It's pretty standard
assert isinstance(X, pd.core.frame.DataFrame), f'RenamingTransformer.transform expected Dataframe but got {type(X)} instead.'


###But now you have a design decision

Here is the second assert from `MappingTransformer`. If we did not supply it, the `replace` method would just ignore bogus columns.
<pre>
    assert self.mapping_column in X.columns.to_list(), f'MappingTransformer.transform unknown column {self.mapping_column}'
</pre>
So we are being strict. If you supply a bogus column (maybe because of typo), you will get an error from the assert. Let's say we want to be just as strict with `RenamingTransformer`. If you supply a bogus column, produce an assert error.

<img src='https://www.dropbox.com/s/8x575mvbi1xumje/cash_line.png?raw=1' height=3 width=500><br>
<img src='https://www.gannett-cdn.com/-mm-/56cbeec8287997813f287995de67747ba5e101d5/c=9-0-1280-718/local/-/media/2018/02/15/Phoenix/Phoenix/636542954131413889-image.jpg' height=50 align=center>  Write the assert statement that checks for bogus columns. It's a little bit tricky.

Take a screenshot of your code and put it on the mural site: https://app.mural.co/invitation/mural/cis4236034/1639511781261?sender=u173416b87b3b8c1cbe335790&key=3993cb03-56a9-4fe5-b4d9-6304e32ff459.




In [ ]:
#Might want to play around here until you get condition correct.
test_map_good =  {'URL':'foo', 'Boat':'fum'}  #ok
test_map_bad =  {'URL':'foo', 'Boot':'fum'}  #typo
X = titanic_table  #just so can test code outside of object.

#you should not need a loop for this. Dredge up some of your Discrete Math concepts :)


In [ ]:
def transform(self, X):
  assert isinstance(X, pd.core.frame.DataFrame), f'RenamingTransformer.transform expected Dataframe but got {type(X)} instead.'
  #your assert code below
  

###A warning could be a compromise

You could still compute the residue. But then just print a warning as opposed to crashing out with an assert error. So at least 3 design choices: (1) ignore, (2) warn, (3) crash out.

#Challenge 1

Write the `OHETransformer`. I'll give you a test case to check your results. I'll also give you a start

In [ ]:
class OHETransformer(BaseEstimator, TransformerMixin):
  def __init__(self, target_column, dummy_na=False, drop_first=True):  
    self.target_column = target_column
    self.dummy_na = dummy_na
    self.drop_first = drop_first
  
  #fill in the rest below
  

###Test it out

In [ ]:
#don't change this code
ohe = OHETransformer(target_column='Joined')
X1_df = ohe.fit_transform(titanic_features)
assert set(['Joined_Cherbourg', 'Joined_Queenstown', 'Joined_Southampton']) - set(X1_df.columns.to_list())==set()

Put it in pipeline and try it out.

In [ ]:
from sklearn.pipeline import Pipeline

#first define the pipeline (but do not invoke it)
titanic_transformer = Pipeline(steps=[
    ('gender', MappingTransformer('Gender', {'Male': 0, 'Female': 1})),
    ('class', MappingTransformer('Class', {'Crew': 0, 'C3': 1, 'C2': 2, 'C1': 3})),
    #add your new ohe step below

    ], verbose=True)

#now invoke it
transformed_df = titanic_transformer.fit_transform(titanic_features)

In [ ]:
transformed_df.head()

<img src='https://www.dropbox.com/s/6ojcjzyitfnwsw3/Screen%20Shot%202021-12-16%20at%2011.30.25%20AM.png?raw=1' height=200>

#Challenge 2

We are missing an early operator, one that drops columns. My preference is to put this operator first in the pipeline. Then follow with mapping and ohe.

Go ahead and build a transformer for dropping columns. I'll give you a start.

Note I have given the choice of either dropping or keeping. The user can use what is easiest depending on whether small number to keep or small number to drop.

In [ ]:
class DropColumnsTransformer(BaseEstimator, TransformerMixin):
  def __init__(self, column_list, action='drop'):
    assert action in ['keep', 'drop'], f'DropColumnsTransformer action {action} not in ["keep", "drop"]'
    assert isinstance(column_list, list), f'DropColumnsTransformer expected list but saw {type(column_list)}'
    self.column_list = column_list
    self.action = action

  #fill in rest below


In [ ]:
#test here - don't change
col_list = ['Age', 'Gender', 'Class', 'Joined', 'Married',  'Fare']
keepers = DropColumnsTransformer(col_list, 'keep')

transformed_df_keep = keepers.fit_transform(titanic_features)
assert set(col_list)-set(transformed_df_keep.columns.to_list())==set()

In [ ]:
transformed_df_keep.head()

In [ ]:
#test here - don't change
col_list = ['Name', 'Bio', 'Occupation', 'Class/Dept', 'Cabin', 'Boat', 'Nationality', 'URL']
droppers = DropColumnsTransformer(col_list, 'drop')
transformed_df_drop = droppers.fit_transform(titanic_features)
assert not set(transformed_df_drop.columns.to_list()).intersection(set(col_list))  #they are gone

In [ ]:
transformed_df_drop.head()

#Challenge 3

Add a drop step to the pipeline as first step.

In [ ]:

titanic_transformer = Pipeline(steps=[
    #add drop step below
    
    ('gender', MappingTransformer('Gender', {'Male': 0, 'Female': 1})),
    ('class', MappingTransformer('Class', {'Crew': 0, 'C3': 1, 'C2': 2, 'C1': 3})),
    ('ohe', OHETransformer(target_column='Joined'))
    ], verbose=True)

#now invoke it
transformed_df = titanic_transformer.fit_transform(titanic_features)

In [ ]:
transformed_df.head()

###What I see

<img src='https://www.dropbox.com/s/fsac9f3vvgw3q0z/Screen%20Shot%202021-09-01%20at%2010.28.55%20AM.png?raw=1'>

#Challenge 4

Build a pipeline for the customer table. As reminder, here it is.




In [ ]:
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQPM6PqZXgmAHfRYTcDZseyALRyVwkBtKEo_rtaKq_C7T0jycWxH6QVEzTzJCRA0m8Vz0k68eM9tDm-/pub?output=csv'

In [ ]:
import pandas as pd
customers_df = pd.read_csv(url)

In [ ]:
customer_features = customers_df.drop(columns='Rating')
customer_features.head()

###Here are steps

* Drop ID.

* There are 4 string feature-columns. Decide how to view each, ordinal or nominal. Then use appropriate transformer.


You will also have to figure out what the possible categorical values are for each string column. There is a pandas method to tell you this. In essence there is a pandas method for everything! It's just figuring out what it is that is the trick :) Google around.

In [ ]:
#figure out how to get unique values in each string column
print(customers_df.apply(lambda col: col.unique()))

In [ ]:
#I'll give you a start
customer_transformer = Pipeline(steps=[
    #fill in the steps on your own

    ], verbose=True)

In [ ]:
transformed_customer_df = customer_transformer.fit_transform(customer_features)
transformed_customer_df.head()

###Your target

<img src='https://www.dropbox.com/s/e8fiqaq6yxwwexw/Screen%20Shot%202021-09-29%20at%201.22.23%20PM.png?raw=1'>